#### import 및 기본변수

In [ ]:
import pandas as pd
import numpy as np
import re, os, functions
from os.path import join
import warnings
warnings.simplefilter("ignore")

debt_dtype = {'채무자키':str, '타채무자키':str, '담당자키':str, '관리자비고':str}
account_dtype = {'채무자키':str, '계좌키':str, '타채무자키':str}
grt_dtype = {'채무자키':str, '계좌키':str, '타채무자키':str, '보증인키':str}
rehabilitation_dtype = {'채무자키':str, '계좌키':str, '분납키':str, '사건키':str, '신고계좌':str, '입금계좌':str}
credit_dtype = {"채무자키":str, "계좌키":str, '보증인키':str, '신용회복키' : str, "계좌번호":str, "심의차수":str, "변제금수취계좌":str}
event_dtype = {'채무자키':str, '법조치키':str, '계좌키':str, '관련법조치키':str, '법취하키':str, '타법조치키':str, '타채무자키':str, '관할법원코드':str}
deposit_dtype = {'채무자키':str, '입금키':str, '계좌키':str, '계좌번호':str, '입금고정키':str, '타채무자키':str}
reduction_dtype = {'채무자키':str, '계좌키':str, '감면키':str}
installment_dtype = {'채무자키':str, '계좌키':str, '분납키':str}

##################################
company = "솔림"      # 솔림 or 대성
basedate = "250824" 
previous_cutoff = "250731" # 계좌 
wd = join(r"D:\3.자산\전산 dataset", company)
##################################
ext = ".xlsx"  
read_dir = join(wd, basedate)
write_dir = join(wd, basedate, "parquet")
previous_read_dir = join(wd, previous_cutoff, "parquet")

# parquet 폴더 만들기
if not os.path.exists(write_dir) :
    os.makedirs(write_dir)


# 작업전 할일 ################################
"""
모든 데이터 다운받기 전에
아래 내용 사전점검 할 것! 이것만 해둬도 파일 다운 받고 기준데이터 만들어도 될듯?
"""
print('신복 특별면책, 반송 확인 후 전산 바로 수정!')
print('개회 엑셀 받은 후 최근 면책건 채무/보증인 상태값만 바로 수정(담당자x)')
print('채무자 다운 후 1. 법인의 채무상태 확인 및 수정-법인도 채무상태로 일단 새채무상태 작성함')
print('계좌 다운 후 2. 담당자 종결, 매각, 환매, ds인데 종결일 없는 건 확인 및 수정')



# 1) object 컬럼 → str
# for col in df.select_dtypes(include='object').columns:
#     df[col] = df[col].astype(str)

신복 특별면책, 반송 확인 후 전산 바로 수정!
개회 엑셀 받은 후 최근 면책건 채무/보증인 상태값만 바로 수정(담당자x)
채무자 다운 후 1. 법인의 채무상태 확인 및 수정-법인도 채무상태로 일단 새채무상태 작성함
계좌 다운 후 2. 담당자 종결, 매각, 환매, ds인데 종결일 없는 건 확인 및 수정


#### 개인회생

In [ ]:
# 엑셀 읽기, 정렬(사건-컷오프-채권번호), parquet 저장
filename = functions.키워드로파일명찾기(read_dir, "개인회생새창", "기준데이터", 전체경로=False).split('.')[0]
df = pd.read_excel(join(read_dir, filename+".xlsx"), dtype=rehabilitation_dtype).fillna("")

# 진행내용 표시 했는지 체크
if df.loc[0,"진행내용"] == "" :
    print("개인회생 엑셀다운로드시 우클릭후 '1.채권자목록,일반내용,진행내용 표시'를 선택해야 합니다.")

# parquet 저장
functions.to_parquet(df, join(write_dir, filename+".parquet"))

#### 전산 raw_data

In [ ]:
#########################################
세부주소 = False # 상세주소 제외할지 모두 포함할지(기본값 False)
# 초본 > 자택 > 직장 > 기타 우선순위 적용은 항상 한다.
파일검색어 = { # [타입, 제외키워드]
         
    # "입금조회.*당월" : [deposit_dtype,""],
    
    # "계좌조회" : [account_dtype,""],
    # "보증인새창" : [grt_dtype,"새보증인상태"],
    
    # "채무자조회" : [debt_dtype,""],
    # "법조치조회" : [event_dtype,""],
    
    "감면조회" : [reduction_dtype,""],
    # "약정분납새창" : [installment_dtype,"기준데이터"]
}
#########################################

# 이전계좌 (종결확인)
account_previous = pd.read_pickle(functions.키워드로파일명찾기(join(previous_read_dir), 포함키워드="계좌조회새창", 전체경로=True))


# 계좌, 보증인새창 읽을 때면 채무상태 누락건 없는지 미리 확인
if ("계좌조회" in 파일검색어.keys()) or ("보증인새창" in 파일검색어.keys()) :
    # 상태정리파일
    path_상태정리 = r"D:\3.자산\전산 dataset\상태정리.xlsx"
    상태정리 = pd.read_excel(path_상태정리, sheet_name="상태", header=1)
    메모정리 = pd.read_excel(path_상태정리, sheet_name="메모")
    담당자정리 = pd.read_excel(path_상태정리, sheet_name="담당자")

    
for i, v in 파일검색어.items() : 
    print(i, '파일변환시작')
    filename = os.path.splitext(functions.키워드로파일명찾기(read_dir, i, v[1],전체경로=False))[0]
    df = pd.read_excel(join(wd, basedate, filename+ext), dtype = v[0]).fillna("")
    
    # 1) 상태정리에 없는 상태값 확인(있으면 상태정리 파일 업데이트 해야! 수정건 있으면 break)
    # 2) 담당자와 메모가 종결값인데 종결일 없는 경우
    if re.search("계좌|보증인", i) :
        if re.search("계좌", i) : 
            상태칼럼명 = "채무상태"
            출력cols = ["채무자키","계좌키", 상태칼럼명, "담당자","종결일"]
            
            # 종결해제건 확인 -----
            전달종결건 = account_previous[account_previous.종결일!=""]["계좌키"]
            종결해제건 = df[(df.종결일=="") & df.계좌키.isin(전달종결건)][출력cols]
            if not 종결해제건.empty :
                functions.display_with_explain(종결해제건, "종결해제건")
            
            
        else : 
            상태칼럼명 = "보증인상태"
            출력cols = ["채무자키","계좌키","보증인키", 상태칼럼명, "담당자","종결일"]
        
        미종결조건 = (df["종결일"]=="") # fillna('')했음
        
        # 상태정리에 없는 상태값 확인
        상태수정할건 = df[미종결조건 & (~df[상태칼럼명].str.replace("_","").isin(상태정리.채무상태)|(df[상태칼럼명]==""))]
        if not 상태수정할건.empty : 
            functions.display_with_explain(상태수정할건[출력cols], "상태정리에 없는 상태값 확인")

    
    # 채무자 관리자비고(새채무자키) 검사
    if re.search("채무자", i) :
        if len(df.query("관리자비고==''")) > 1 :
            print("새채무자키 부여 필요")
            print(len(nothing)) # 오류를 위한 코드
        
        # 예수금제외
        df.drop(df[(df.성명=="예수금")].index, inplace=True)
        df.reset_index(drop=True, inplace=True)
        
        # 채무자 주소 보정  
        cols = ["초본주소", "초본번지인", "초본우편번호", "자택주소","자택번지인","자택우편번호","직장주소","직장번지인", "직장우편번호","기타주소", "기타번지인", "기타우편번호"]
        df[cols] = df[cols].applymap(lambda x: x.strip() if isinstance(x, str) else x)


        # 주소채우기 : 초본이 없는 경우 자택 > 직장 > 기타 주소를 초본칸에 넣기
            # values를 붙이거나 to_numpy()를 붙여야함.(안그럼 정렬과 인덱스유지 사이의 충돌로 na값이 입력됨)
            # 이제 isna()조건은 없어도 되지만 혹시 몰라서 추가함
        mask = ((df.초본주소=="") | (df.초본주소.isna()))
        df.loc[mask,["초본주소", "초본번지인", "초본우편번호"]] = df.loc[mask, ["자택주소", "자택번지인", "자택우편번호"]].to_numpy()
        mask = ((df.초본주소=="") | (df.초본주소.isna()))
        df.loc[mask,["초본주소", "초본번지인", "초본우편번호"]] = df.loc[mask, ["직장주소", "직장번지인", "직장우편번호"]].to_numpy()
        mask = ((df.초본주소=="") | (df.초본주소.isna()))
        df.loc[mask,["초본주소", "초본번지인", "초본우편번호"]] = df.loc[mask, ["기타주소", "기타번지인", "기타우편번호"]].to_numpy()
    
    # 계좌 예수금 제거
    if re.search("계좌", i) :
        df.drop(df[(df.채무자명=="예수금")].index, inplace=True)
        df.reset_index(drop=True, inplace=True)
        
    # 법조치 법원사건번호 없는 것 제거
    if re.search('법조치', i) :
        index =  df[(df.관할법원=="0") | (df.관할법원=="") | (df.사건번호=="0") | (df.사건번호=="")].index
        df.drop(index, inplace=True)
        df.reset_index(drop=True, inplace=True)
        # 사건번호 역순정렬
        df.sort_values('사건번호', ascending=False, inplace=True)
    
    # parquet 저장
    functions.to_parquet(df, join(write_dir, filename+".parquet"), engine="pyarrow")

ModuleNotFoundError: No module named 'numpy._core.numeric'

#### 신용회복 계좌별진행상황

In [ ]:
# 계좌별 진행상황 파일 읽기
########################################################
약식병합 = False ############# 금요일(일요일)이 컷오프가 아니라면 약식병합해야-------
basedate = basedate # "250423" # 정식 업로드결과 파일 날짜 컷오프일에 맞춰 수정
company = company  # 솔림 or 대성
########################################################

nauri_dtype = {"채무자키":str, "계좌키":str, '보증인키':str, '신용회복키' : str, "계좌번호":str, "심의차수":str, "변제금수취계좌":str}
path_base = r"D:\3.자산\신용회복\신용회복 전체데이터\계좌별 진행상황"

# parquet 폴더 없으면 만들기
write_path_base = join(path_base, "parquet")
if not os.path.exists(write_path_base) :
    os.mkdir(write_path_base)

# 읽을 파일명
suffix = "통합" if company!="대성" else "대성"

filename = basedate + "_계좌별 진행상황 조회_" + suffix

filename_정식 = filename + "_업로드결과"
filename_약식 = filename + "_약식_업로드결과"

# 계좌별 진행상황 조회 파일 읽기 및 parquet 저장
    # 정식
정식_ori = pd.read_excel(join(path_base, filename_정식+".xlsx" ), dtype=nauri_dtype).fillna("")
functions.to_parquet(정식_ori, join(write_path_base, filename_정식+".parquet"))
    # 약식
if 약식병합 : 
    약식_ori = pd.read_excel(join(path_base, filename_약식+".xlsx"), dtype=nauri_dtype).fillna("")
    functions.to_parquet(약식_ori, join(write_path_base, filename_약식+".parquet"))

# 이건 기준데이터 아니니 파일 옮길 필요 없음.

### 기타

#### 단일파일

#### 폴더

In [ ]:
##########################################################
path_read = r"D:\3.자산\전산 dataset\솔림\기간축적데이터\신용회복"
dtype_to_use = credit_dtype
##########################################################
p_extension = re.compile('xlsx$', re.I)
file_list = [f.name for f in os.scandir(path_read) if f.is_file() and p_extension.search(f.name)]

for f in file_list :
    print("변환시작 :",f)
    fn = os.path.splitext(f)[0]
    df = pd.read_excel(join(path_read, f), dtype=dtype_to_use)
    functions.to_parquet(df, join(path_read, "parquet", fn+".parquet"), engine="pyarrow")

변환시작 : 감면율로그_~250630.xlsx
변환시작 : 상환후잔액로그_~250630.xlsx
변환시작 : 신청인진행상태로그_~250630.xlsx
변환시작 : 접수번호로그_~250630.xlsx
변환시작 : 합의서체결일로그_~250630.xlsx
